In [1]:
from utils.load_models import *
from utils.novelty_serve import *

In [2]:
model, lang, model_conf = load_han_novelty("NOV-211")
model_conf.num_sent = 100
model_dict = {"model": model.cuda(), "lang": lang, "model_conf": model_conf}

In [3]:
import pandas as pd

data = pd.read_csv("dlnd_data.csv")

In [5]:
row = data.iloc[1]
target_text = row.target_text
source_text = row.source
true_class = row.DLA

In [6]:
opt, src_att, trg_att = map_attention(source_text, target_text, model_dict)

In [7]:
import pandas as pd
import numpy as np
import html
import random
from IPython.core.display import display, HTML
from IPython.display import IFrame
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


def colorize(words, color_array):
    cmap=matplotlib.cm.Reds
    template = '<span class="barcode"; style="color: black; background-color: {}">{}</span>'
    colored_string = ''
    for word, color in zip(words, color_array):
        color = matplotlib.colors.rgb2hex(cmap(color)[:3])
        colored_string += template.format(color, '&nbsp' + word + '&nbsp')
    return colored_string




In [8]:
def plot(sent,attn):
    source = nltk.sent_tokenize(sent)
    if torch.is_tensor(attn):
        attn = attn.detach().cpu().numpy().flatten()
    attn_weights = 3*attn[0:len(source)]
    s = colorize(source,attn_weights)
    
    # or simply save in an html file and open in browser
    with open('colorize.html', 'w') as f:
        f.write(s)


In [9]:
row = data.iloc[5349]
target_text = row.target_text
source_text = row.source
true_class = row.DLA

In [10]:
opt, src_att, trg_att = map_attention(source_text, target_text, model_dict)

In [11]:
plot(source_text,src_att)
IFrame('./colorize.html',width=1200,height=400)



In [12]:
plot(target_text,trg_att)
IFrame('./colorize.html',width=1200,height=400)
